In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import implicit
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from numpy.core.umath_tests import inner1d

/home/raghavjajodia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  if __name__ == '__main__':


In [42]:
full_data = pd.read_csv('data/final_data.csv', index_col = 0)

/home/raghavjajodia/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [139]:
class cdfTransformer(object):
    
    transformerSeries = {}
    
    def __init__(self, useridcolname, itemidcolname, ratingcolname):
        self.userid_colname = useridcolname
        self.itemid_colname = itemidcolname
        self.rating_colname = ratingcolname
    
    def getLowerOrEqualIndex(self,playtimeList, playtime):
        if playtime < playtimeList[0]:
            return 0
        ans = 0
        low = 0
        high = len(playtimeList) - 1
        while(low<=high):
            mid = low + (high-low)//2
            if playtime > playtimeList[mid]:
                ans = mid
                low = mid + 1
            elif playtime == playtimeList[mid]:
                return mid
            else:
                high = mid - 1
        return ans      
    
    def getNearestCdf(self,appid, playtime):
        playtimeList = self.transformerSeries[appid].index
        bestpos = self.getLowerOrEqualIndex(playtimeList, playtime)
        return self.transformerSeries[appid].iloc[bestpos]
    
    def fitTransform(self,tupledata):
        grouped1 = tupledata.groupby([self.itemid_colname,self.rating_colname]).count()
        grouped2 = grouped1.groupby(level=[0]).cumsum()
        grouped3 = grouped2.groupby(level = [0]).max()
        withcdf = grouped2/grouped3
        self.transformerSeries = pd.Series(withcdf[self.userid_colname],index=withcdf.index)
        withcdf_df = withcdf.reset_index(level=[0,1])
        withcdf_df.rename(columns={self.userid_colname:self.rating_colname}, inplace=True)
        finaltuple = pd.merge(withcdf_df,tupledata, on=[self.itemid_colname,self.rating_colname],how='inner',suffixes=('_newdf',''))
        return finaltuple

    def Transform(self,tupledata):
        ansdata = tupledata.groupby([self.itemid_colname,self.rating_colname]).count().reset_index()
        ansdata.drop(self.userid_colname, inplace = True, axis = 1)
        ansdata[self.rating_colname] =  ansdata.apply(lambda x: self.getNearestCdf(x[self.itemid_colname],x[self.rating_colname]), axis = 1)
        return ansdata

In [60]:
def train_test_split(dff, split_ratio = 0.8):
    
    tr_sample = dff.groupby('sid').apply(lambda x:x.sample(frac = 0.8))
    tr_sample_index = list(zip(*tr_sample.index))[1]
    te_sample = dff[(dff.index.isin(tr_sample_index) == False)]
    
    tr_sample.index.rename(['id', 'appid_level'], inplace=True)
    tr_sample.reset_index(drop = True, inplace = True)
    te_sample.reset_index(drop = True, inplace = True)
    
    return tr_sample, te_sample

In [34]:
N = full_data['appid'].nunique()
M = full_data['steamid'].nunique()

#Number of hyperparameter, hyperparameter
K = 10
N,M,K
#-------------------------
# LOAD AND PREP THE DATA
#-------------------------
 
#  
 # Convert artists names into numerical IDs
 #data['user_id'] = data['user'].astype("category").cat.codes
 #data['artist_id'] = data['artist'].astype("category").cat.codes
 
 # Create a lookup frame so we can get the artist names back in 
 # readable form later.
 #item_lookup = data[['artist_id', 'artist']].drop_duplicates()
 #item_lookup['artist_id'] = item_lookup.artist_id.astype(str)
 
 #data = data.drop(['user', 'artist'], axis=1)

(12982, 13845, 10)

In [204]:
class matrixFactorizer(object):
    
    def __init__(self, latentFactors = 3, max_iterations = 20, reg = 0.01):        
        self.numLatent = latentFactors
        self.max_iterations = max_iterations
        self.reg = reg
    
    def fit(self,train_ratingsbyuser, train_ratingsbyitem, total_users, total_items):
        self.U = np.random.randn(total_users, self.numLatent) / self.numLatent
        self.V = np.random.randn(self.numLatent, total_items) / self.numLatent
        
        for t in range(self.max_iterations):
          # update U
          for i in range(total_users):
            if i in train_ratingsbyuser:
              matrix = np.zeros((K, K)) + self.reg*np.eye(K)
              vector = np.zeros(K)
              for j, r in train_ratingsbyuser[i]:
                matrix += np.outer(self.V[:,j], self.V[:,j])
                vector += (r)*V[:,j]
              self.U[i,:] = np.linalg.solve(matrix, vector)

          # update V
          for j in range(total_items):
            if j in train_ratingsbyitem:
              matrix = np.zeros((K, K)) + self.reg*np.eye(K)
              vector = np.zeros(K)
              for i, r in train_ratingsbyitem[j]:
                matrix += np.outer(self.U[i,:], self.U[i,:])
                vector += (r)*U[i,:]
              self.V[:,j] = np.linalg.solve(matrix, vector)
    
    def predict(self, user_u, item_i):
        return inner1d(self.U[user_u,:],np.transpose(self.V)[item_i,:])

In [45]:
def constructCodes(df) :
    """
    Maps 'steamid' to categorical code 'sid'
    Maps 'appid' to categorical code 'aid'
    Returns :
        1. Dataframe with columns 'sid', 'aid', 'playtime_forever'
        2. Dictionary mapping 'steamid' to 'sid'
        3. Dictionary mapping 'sid' to 'steamid'
        4. Dictionary mapping 'appid' to 'aid'
        5. Dictionary mapping 'aid' to 'appid'
    """
    coded_df = df.copy(deep = True)
    coded_df["steamid"] = coded_df["steamid"].astype("category")
    coded_df["appid"] = coded_df["appid"].astype("category")
    coded_df["sid"] = coded_df["steamid"].cat.codes
    coded_df["aid"] = coded_df["appid"].cat.codes
    
    sid_to_steamid = dict(enumerate(coded_df["steamid"].cat.categories))
    aid_to_appid = dict(enumerate(coded_df["appid"].cat.categories))
    steamid_to_sid = {v : k for k, v in sid_to_steamid.items()}
    appid_to_aid = {v : k for k, v in aid_to_appid.items()}
    
    coded_df.drop(["steamid", "appid"], axis = 1, inplace = True)
    
    return(coded_df, steamid_to_sid, sid_to_steamid, appid_to_aid, aid_to_appid)

In [46]:
def constructSparseMatrices(df) :
    """
    Constructs sparse matrices that will be used in ALS optimization
    Input : Dataframe with columns - 'sid', 'aid', 'playtime_forever'
    
    Returns :
        1. User x Item Sparse Matrix
        2. Item x User Sparse Matrix
    """
    data_useritem = sparse.csr_matrix((df["playtime_forever"], (df["sid"], df["aid"])))
    data_itemuser = sparse.csr_matrix((df["playtime_forever"], (df["aid"], df["sid"])))
    
    sid_unique = df["sid"].nunique()
    aid_unique = df["aid"].nunique()
    
    assert data_useritem.shape == (sid_unique, aid_unique)
    assert data_itemuser.shape == (aid_unique, sid_unique)
    
    return(data_useritem, data_itemuser)

In [133]:
def trainModel(data, factors, epochs, conf_func, alpha, lmbda) :
    """
    Builds and trains Implicit Matrix Factorization model.
    Input :
        1. data - Item x User Sparse Matrix
        2. factors - Number of latent factors
        3. epochs - Number of iterations of ALS over the training data
        4. conf_func - Confidence function
        5. alpha - Confidence parameter
        6. lmbda - Regularization parameter
        
    Output : model
    """
    model = implicit.als.AlternatingLeastSquares(factors = factors, regularization = lmbda, iterations = epochs)
    if conf_func == "linear" :
        model.fit(alpha * data)
    else :
        print("{} is not a valid choice for conf_func. Choose one of the following : 'linear'".format(conf_func))
        return(None)
    return(model)

In [132]:
def evaluateValidationLoss(item_factors, user_factors, testtuples, itemidcolname, useridcolname, ratingcolname):
    """
    Input :
        1. item_factors - items * latentsize
        2. user_factors - users * latentsize
    Output : validationloss
    """
    predictions = inner1d(item_factors[testtuples[itemidcolname],:],user_factors[testtuples[useridcolname],:])
    return np.sqrt((((testtuples[ratingcolname] - predictions)**2).dot(np.ones(testtuples.shape[0])))/testtuples.shape[0])

In [56]:
coded_data, steamidtosid, sidtosteamid, appidtoaid, aidtoappid = constructCodes(full_data)

In [67]:
train,test = train_test_split(coded_data)
cdftransformerobj = cdfTransformer()
traintuples = cdftransformerobj.fitTransform(train)
testtuples = cdftransformerobj.Transform(test)

In [74]:
testtuples.head()

,aid,playtime_forever,playtime_cdf
0,0,1,0.096587
1,0,2,0.113652
2,0,3,0.130717
3,0,4,0.144027
4,0,5,0.152218


In [76]:
traintuples.drop('playtime_forever', axis = 1, inplace=True)
traintuples.head()

,aid,playtime_cdf,sid
0,0,0.096587,73
1,0,0.096587,74
2,0,0.096587,75
3,0,0.096587,76
4,0,0.096587,77


In [77]:
traintuples.rename(columns={"playtime_cdf":"playtime_forever"}, inplace=True)

In [81]:
testtuples = pd.merge(testtuples,test, on=['aid','playtime_forever'],how='inner',suffixes=('_newdf',''))
testtuples.drop('playtime_forever', axis = 1, inplace = True)
testtuples.rename(columns = {"playtime_cdf":"playtime_forever"}, inplace = True)
testtuples.head()

,aid,playtime_forever,sid
0,0,0.096587,1
1,0,0.096587,30
2,0,0.096587,58
3,0,0.096587,94
4,0,0.096587,99


In [116]:
predictions = inner1d(model.item_factors[testtuples['aid'],:],model.user_factors[testtuples['sid'],:])

0.4449343065072283

In [99]:
testtuples.shape

(660690, 3)

In [83]:
traintuples['aid'].nunique(), traintuples['sid'].nunique(), testtuples['aid'].nunique(), testtuples['sid'].nunique()

(12982, 13845, 12866, 13845)

In [84]:
full_data['appid'].nunique()

12982

In [86]:
user_item_mat, item_user_mat = constructSparseMatrices(traintuples)

In [88]:
user_item_mat.shape

(13845, 12982)

In [140]:
factors = [5,10,50,100]
epochs = [10,50,100]
lambdas = [0.001,0.01,0.1,1]

allModels = {}
bestLoss = 10000000

for factor in factors:
    for epoch in e pochs:
        for lamb in lambdas:
            tempmodel = trainModel(item_user_mat,factor, epoch,'linear',1,lamb)
            valLoss = evaluateValidationLoss(tempmodel.item_factors, tempmodel.user_factors, testtuples, 'aid', 'sid', 'playtime_forever')
            key = str(factor) + "_" + str(epoch) + "_" + str(lamb)
            allModels[key] = tempmodel
            
            if(valLoss < bestLoss):
                bestLoss = valLoss
                bestConfiguration = (factor, epoch, lamb)

100%|██████████| 10.0/10 [00:02<00:00,  3.73it/s]
100%|██████████| 10.0/10 [00:02<00:00,  3.73it/s]
100%|██████████| 10.0/10 [00:02<00:00,  3.78it/s]
100%|██████████| 10.0/10 [00:02<00:00,  3.80it/s]
100%|██████████| 50.0/50 [00:16<00:00,  2.90it/s]
100%|██████████| 50.0/50 [00:17<00:00,  2.84it/s]
100%|██████████| 50.0/50 [00:17<00:00,  2.89it/s]
100%|██████████| 50.0/50 [00:17<00:00,  2.89it/s]
100%|██████████| 100.0/100 [00:36<00:00,  2.91it/s]
100%|██████████| 100.0/100 [00:34<00:00,  2.87it/s]
100%|██████████| 100.0/100 [00:34<00:00,  2.78it/s]
100%|██████████| 100.0/100 [00:35<00:00,  2.86it/s]
100%|██████████| 10.0/10 [00:03<00:00,  2.71it/s]
100%|██████████| 10.0/10 [00:03<00:00,  2.78it/s]
100%|██████████| 10.0/10 [00:03<00:00,  2.76it/s]
100%|██████████| 10.0/10 [00:03<00:00,  2.66it/s]
100%|██████████| 50.0/50 [00:17<00:00,  2.74it/s]
100%|██████████| 50.0/50 [00:17<00:00,  2.80it/s]
100%|██████████| 50.0/50 [00:19<00:00,  2.79it/s]
100%|██████████| 50.0/50 [00:18<00:00,  2.

In [91]:
model = trainModel(item_user_mat, 10, 10, 'linear', 1, 0.1)

100%|██████████| 10.0/10 [00:02<00:00,  4.17it/s]


In [143]:
bestConfiguration

(50, 10, 0.1)

In [134]:
evaluateValidationLoss(model.item_factors, model.user_factors, testtuples, 'aid', 'sid','playtime_forever')

0.4449343065072283

In [163]:
temptraintuples = traintuples
temptraintuples['tup'] = list(zip(traintuples['sid'], traintuples['playtime_forever']))
ratingsbyitem = temptraintuples.groupby('aid')['tup'].apply(list).to_dict()

temptraintuples2 = traintuples
temptraintuples2['tup'] = list(zip(temptraintuples2['aid'], temptraintuples2['playtime_forever']))
ratingsbyuser = temptraintuples2.groupby('sid')['tup'].apply(list).to_dict()

In [205]:
matFact1 = matrixFactorizer(latentFactors=10)
matFact1.fit(ratingsbyuser, ratingsbyitem, traintuples['sid'].nunique(), traintuples['aid'].nunique())

In [206]:
evaluateValidationLoss(np.transpose(matFact1.V), matFact1.U, testtuples, 'aid', 'sid', 'playtime_forever')

701.752169243928

In [209]:
matFact1.predict(1000,1100)

3001.6589134723595

In [ ]:
latentfactors = [5,10,20,50]
maxiterations = [10,20,50]
regularizations = [0.1,0.01,0.001]

